In [115]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

df = pd.read_csv("billboard100_data.csv")
df_cluster = pd.read_csv("clustered_data.csv")

In [116]:
df['title'] = df['title'].map(str.lower) # converting the entries to lower
df['author'] = df['author'].map(str.lower) # converting the entries to lower

,title,author
0,flowers,miley cyrus
1,kill bill,sza
2,last night,morgan wallen
3,unholy,sam smith & kim petras
4,creepin',"metro boomin, the weeknd & 21 savage"
...,...,...
95,yandel 150,yandel & feid
96,heart to heart,mac demarco
97,next thing you know,jordan davis
98,never gonna not dance again,p!nk


In [125]:
x = input('Enter a song you like: ')
x_artist = df.loc[df['title'].map(str.lower)  == x.lower(), 'author'] # artist corresponding to song x
if x.lower() in df.values:   # converts the entered x into all lower case. When x is in the billboard100
        rec_song = df['title'].sample().to_string(index=False) # recommendation song. I just removed the indexes as they are not required as the output  
        rec_artist = df.loc[df['title'] == rec_song, 'author'].to_string(index=False) # artist of the recommended song
        print("You might like this song: " + rec_song.title() + " by " + rec_artist.title())  # .title() makes the first alphabet in every word to upper   
else: 
    results = sp.search(q=x,limit=3,market="GB")
    track_id=track_id=results["tracks"]["items"][0]["id"]
    x_af = sp.audio_features(track_id) 
    X_af = pd.DataFrame(x_af)
    X_af_num = X_af.drop(['key', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)
    df_spotify = pd.read_csv("spotify_data.csv")
   
    X = df_spotify.drop(['id', 'duration_ms'], axis=1) # spotify data without id and duration
    scaler = StandardScaler()
    scaler_fitted = scaler.fit(X) # we need scaler_fitted for transforming the song which is to be inputed 
    X_scaled = scaler_fitted.transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns) # scaled the numerical feature data X

    X_af_num_scaled = scaler_fitted.transform(X_af_num) # scaled according to the parameters from scaler_fitted (which you get from your data X)
    
    kmeans = KMeans(n_clusters=4, random_state=1234)
    kmeans.fit(X_scaled_df.values)
    song_cluster = kmeans.predict(X_af_num_scaled) # cluster of the inputed song
    
    if song_cluster == 0:  
        mask = df_cluster['cluster'] == 0
        selected_rows = df_cluster[mask]
        random_row = selected_rows.sample()
        track_id = random_row['id'].to_string(index=False)
        track_id
        track = sp.track(track_id)
        print('You might like this song: ' + track["album"]["name"]+ ' by ' + track["album"]["artists"][0]["name"])  
    elif song_cluster == 1:    
        mask = df_cluster['cluster'] == 1
        selected_rows = df_cluster[mask]
        random_row = selected_rows.sample()
        track_id = random_row['id'].to_string(index=False)
        track_id
        track = sp.track(track_id)
        print('You might like this song: ' + track["album"]["name"]+ ' by ' + track["album"]["artists"][0]["name"]) 
    elif song_cluster == 2:
        mask = df_cluster['cluster'] == 2
        selected_rows = df_cluster[mask]
        random_row = selected_rows.sample()
        track_id = random_row['id'].to_string(index=False)
        track_id
        track = sp.track(track_id)
        print('You might like this song: ' + track["album"]["name"]+ ' by ' + track["album"]["artists"][0]["name"])   
    else:
        mask = df_cluster['cluster'] == 3
        selected_rows = df_cluster[mask]
        random_row = selected_rows.sample()
        track_id = random_row['id'].to_string(index=False)
        track_id
        track = sp.track(track_id)
        print('You might like this song: ' + track["album"]["name"]+ ' by ' + track["album"]["artists"][0]["name"])   

Enter a song you like: flowerzz
You might like this song: Lionheart (Fearless) by Joel Corry


In [119]:
# Just want to play the inputed song
from IPython.display import IFrame

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )